#### Leetcode 53: 最大子序和问题
给定一个整数数组 nums ，找到一个具有最大和的连续子数组（子数组最少包含一个元素），返回其最大和。
思路：
1. 状态定义：dp[i]为以nums[i]结尾的最大子序和
2. 状态转移方程：对于nums[i]来说有两种情况：
    a.之前的子序列连续 dp[i] = dp[i-1] + nums[i]
    b.从自己开始算  dp[i] = nums[i] (即前面最大和为负数的情况)
    * 可以由反证法得到，最大子序和必定是上述两种情况之一
    * 考虑是否正确的思考方法类似于递归，如果某种思路能适应任何一环，那么大概率加上边界条件之后就能适用于所有，当然刚刚开始做最好还是先想清楚再做
3. basecase：dp[0] = nums[0]

```python
class Solution:
    def maxSubArray(self, nums: List[int]) -> int:
        if nums == []:
            return None
        dp = [None for i in range(len(nums))]
        dp[0] = nums[0]
        result = dp[0]
        for i in range(1,len(nums)):
            if dp[i-1] < 0:
                dp[i] = nums[i]
            else:
                dp[i] = dp[i-1] + nums[i]
            if dp[i] > result:
                result = dp[i]
        return result
```

优化后空间复杂度可减小为O(1)
```python
class Solution:
    def maxSubArray(self, nums: List[int]) -> int:
        if nums == []:
            return None
        prev = nums[0]
        curr = None
        result = prev
        for i in range(1,len(nums)):
            if prev < 0:
                curr = nums[i]
            else:
                curr = prev + nums[i]
            if curr > result:
                result = curr
            prev = curr
        return result
```

#### Leetcode: 面试题 17.24. 最大子矩阵
- 给定一个正整数、负整数和 0 组成的 N × M 矩阵，编写代码找出元素总和最大的子矩阵。
- 返回一个数组 [r1, c1, r2, c2]，其中 r1, c1 分别代表子矩阵左上角的行号和列号，r2, c2 分别代表右下角的行号和列号。若有多个满足条件的子矩阵，返回任意一个均可。
- 注意：本题相对书上原题稍作改动
```
Example:
输入：
[
   [-1,0],
   [0,-1]
]
输出：[0,1,0,1]
解释：输入中标粗的元素即为输出所表示的矩阵
```

思路：
本题实际上是相当于最长子序列的进阶版，经过分析可得，问题等效为先枚举由i-j行所有矩阵每列单元格之和，然后对这个新矩阵（下面方法中的sums）求最长子序列，记录全局最大值
* 能否简化sums的空间复杂度：不行，因为sums每向下递进一列都需要上一个sums[k]作为参照
原链接：https://leetcode-cn.com/problems/max-submatrix-lcci/solution/zhe-yao-cong-zui-da-zi-xu-he-shuo-qi-you-jian-dao-/
```python
class Solution:
    def getMaxMatrix(self, matrix: List[List[int]]) -> List[int]:
        if matrix is None or matrix == []: return None
        
        sums = [0 for _ in range(len(matrix[0]))]
        width = len(matrix[0])
        height = len(matrix)
        prev = -float("inf")
        max_val = -float("inf")
        ans = [None for _ in range(4)]
        for i in range(0,height):
            sums = [0 for _ in range(width)]
            for j in range(i,height):
                prev = -float("inf")
                for k in range(width):
                    sums[k] += matrix[j][k]
                    if prev < 0:
                        prev = sums[k]
                        lefttop_x = i
                        lefttop_y = k
                    else:
                        prev += sums[k]
                    if prev > max_val:
                        max_val = prev
                        ans = [lefttop_x,lefttop_y,j,k]
        return ans
 ```


#### Leetcode 718: Maximum Length of Repeated Subarray
```
Given two integer arrays A and B, return the maximum length of an subarray that appears in both arrays.

Example 1:

Input:
A: [1,2,3,2,1]
B: [3,2,1,4,7]
Output: 3
Explanation: 
The repeated subarray with maximum length is [3, 2, 1].

 

Note:

    1 <= len(A), len(B) <= 1000
    0 <= A[i], B[i] < 100
```
思路：
首先可以考虑暴力方法求解，列举出所有的A[i:j]与B[i:j]进行比较，时间复杂度为N**3，会超时
```python
class Solution:
    def findLength(self, A: List[int], B: List[int]) -> int:
        if B == [] or B is None:
            return None
        pA = pB = 0
        max_val = 0
        for i in range(len(A)):
            for j in range(len(B)):
                pB = j
                pA = i
                repeat_length = 0
                while pB < len(B) and pA < len(A):
                    if B[pB] == A[pA]:
                        repeat_length += 1
                        pB += 1
                        pA += 1
                    else:
                        if max_val < repeat_length: max_val = repeat_length
                        repeat_length = 0
                        pB += 1
                        pA = i
                if max_val < repeat_length: max_val = repeat_length
        return max_val
```

上述方法效率不高的最大问题在于同一个字串会重复判断多次,例如假如A = [A,R,D,X] B = [A,R,C,S], R这一段在整个过程中会被重复计算若干次。

所以简化问题的核心在于重新设计算法过程或者采用存储的方法减少重复操作的次数，即DP的基本思路。

在这个问题中，可以设dp[i][j] 为 A[j:] 与 B[i:] 重合字串的最长长度，注意两个subarray必须都是以j和i开头。

这么设计的原因在于：
1. 好写状态转移方程，因为A和B的开头字母都是确定的，dp[i][j] = ||A[j] == B[i]|| + dp[i+1][j+1]
2. 只需要从尾到头遍历，就能自动实现bottom-up solution防止遗漏

*PS：注意本题中他采取的那种从i j = 0开始取起的方法，便于initialize

*一开始让dp[i][-1], dp[j][-1] = ||A[i] == B[j]||不行的原因是违背了A[i]和B[j]的定义，以ij开始而不是结束（和我一开始写法的一个错误有关，没理解什么意思就直接忽略）

```python
class Solution:
    def findLength(self, A: List[int], B: List[int]) -> int:
        if B == [] or B is None:
            return None
        max_val = -float("inf")
        len_A = len(A)
        len_B = len(B)
        dp = [[0 for _ in range(len_B+1)] for _ in range(len_A+1)]
        for i in range(len_A-1,-1,-1):
            for j in range(len_B-1,-1,-1):
                dp[i][j] = 1 + dp[i+1][j+1] if A[i] == B[j] else 0
                max_val = max(dp[i][j],max_val)
        return max_val
```